In [69]:
import os 
import numpy as np
import sympy
import cirq
import tensorflow_quantum as tfq
from tqdm import tqdm
import tensorflow as tf

from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

%load_ext autoreload
%autoreload 2


n_qubits = 4
qlr = 0.01
qepochs = 10**3
verbose=0
g=1
J=0
noise=0.0
problem="TFIM"
vqe_handler = VQE(n_qubits=n_qubits, lr=qlr, epochs=qepochs, patience=100,
                  random_perturbations=True, verbose=verbose, g=g, J = J, 
                  problem=problem, noise_model={ "channel":"depolarizing", "channel_params":[0.01], "q_batch_size":10**2})

iid = IdInserter(n_qubits=n_qubits)
Simp = Simplifier(n_qubits=n_qubits)
killer = UnitaryMurder(vqe_handler)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
indexed_circuit = [vqe_handler.n_qubits+k+vqe_handler.number_of_cnots for k in range(vqe_handler.n_qubits)]
vqe_handler.give_circuit(indexed_circuit)

((0, 0): ───Rx(th_0)───

(0, 1): ───Rx(th_1)───

(0, 2): ───Rx(th_2)───

(0, 3): ───Rx(th_3)───,
 ['th_0', 'th_1', 'th_2', 'th_3'],
 {0: 'th_0', 1: 'th_1', 2: 'th_2', 3: 'th_3'})

In [71]:
qbatch, symbols, index_to_symbols = vqe_handler.give_circuit_with_noise(indexed_circuit)
qbatch_resolved=[]
for k in qbatch:
    qbatch_resolved.append(cirq.resolve_parameters(k,{s:0 for s in symbols}))
tfq_circuit = tfq.convert_to_tensor(qbatch_resolved)
expectation = tfq.layers.Expectation()
tfq_layer = expectation(tfq_circuit, operators=tfq.convert_to_tensor([vqe_handler.observable]*vqe_handler.q_batch_size))
averaged_unitaries = tf.math.reduce_mean(tfq_layer, axis=0)
energy = np.squeeze(tf.math.reduce_sum(averaged_unitaries, axis=-1))
noisy_energy.append(energy)

array(-3.98, dtype=float32)

In [73]:
circuit, symbols, idx = vqe_handler.give_circuit(indexed_circuit)
circuit = cirq.resolve_parameters(circuit,{s:0 for s in symbols} )
tfq_circuit = tfq.convert_to_tensor([circuit])
expectation = tfq.layers.Expectation()
tfq_layer = expectation(tfq_circuit, operators=tfq.convert_to_tensor([vqe_handler.observable]))
averaged_unitaries = tf.math.reduce_mean(tfq_layer, axis=0)
energy = np.squeeze(tf.math.reduce_sum(averaged_unitaries, axis=-1))
energy

array(-4., dtype=float32)

In [20]:
pp=0.1*np.arange(1,10)
np.random.choice(range(9), 1, p=pp/np.sum(pp))

array([7])

In [17]:
probs

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])